In [ ]:
# Type how many pages you would like to scrape
a = int(input("Number of Pages to Scrape:"))

In [60]:
from nltk import word_tokenize
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
from scipy.misc import imread
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import lda 
from collections import Counter

In [1]:
import requests
from lxml import etree
import re
import pandas as pd
from bs4 import BeautifulSoup
import time

In [152]:
my_stopwords=set(STOPWORDS)
my_stopwords.update(['MCAP','benefit','hours','day','Good','Great','I',
                     'A','great','good','co','lot','able','The','company',
                     'job','employee','bank','lots','work','people','one','product','working','It','You','will','benefits','us'
                    ,'sorry','much','every','time','If','shifts','know','We','review', 'hear','discuss','contact','Thank','please'
                    ,'management','team','No','There','place','Very'])


In [2]:
headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

In [141]:
#get the original pages for further scraping review page links 
urls=[]
for i in range(1,50):
    urls.append('https://www.glassdoor.ca/Reviews/toronto-reviews-SRCH_IL.0,7_IM976_IP{}.htm'.format(i))

In [142]:
#get each company review pages original link from the company infomation
comp_r_links=[]
company_names=[]
for url in urls:
    res=requests.get(url,headers=headers)
    html=etree.HTML(res.text)
    name=html.xpath('//div[@class=" margBotXs"]/a/text()')
    review_link=html.xpath('//a[@class="eiCell cell reviews"]/@href')
    comp_r_links.append(review_link)
    company_names.append(name)

In [143]:
#create an array of company review links and company names
links=[]
for i in comp_r_links:
    for j in i:
        s='https://www.glassdoor.ca'+j
        links.append(s)
companies=[]
for i in company_names:
    for j in i:
        companies.append(j.strip())

In [144]:
#process each link for next review content scraping
iter_links=[]
for i in links:
    L=i.split('.htm')
    S=L[0]+'_IP{}.htm'
    iter_links.append(S)

In [145]:
all_links=[]
for i in range(len(iter_links)):
    all_links.append([iter_links[i].format(1),iter_links[i].format(2),iter_links[i].format(3),iter_links[i].format(4),
                     iter_links[i].format(5),iter_links[i].format(6),iter_links[i].format(7)])       

In [147]:
all_reviews=[]
for i in all_links:
    all_reviews.append(etree.HTML(requests.get(i[0],headers=headers).text).xpath('//p[@class="mt-0 mb-xsm v2__EIReviewDetailsV2__bodyColor v2__EIReviewDetailsV2__lineHeightLarge  "]/text()')+
                      etree.HTML(requests.get(i[1],headers=headers).text).xpath('//p[@class="mt-0 mb-xsm v2__EIReviewDetailsV2__bodyColor v2__EIReviewDetailsV2__lineHeightLarge  "]/text()')+
                      etree.HTML(requests.get(i[2],headers=headers).text).xpath('//p[@class="mt-0 mb-xsm v2__EIReviewDetailsV2__bodyColor v2__EIReviewDetailsV2__lineHeightLarge  "]/text()')+
                      etree.HTML(requests.get(i[3],headers=headers).text).xpath('//p[@class="mt-0 mb-xsm v2__EIReviewDetailsV2__bodyColor v2__EIReviewDetailsV2__lineHeightLarge  "]/text()')+
                      etree.HTML(requests.get(i[4],headers=headers).text).xpath('//p[@class="mt-0 mb-xsm v2__EIReviewDetailsV2__bodyColor v2__EIReviewDetailsV2__lineHeightLarge  "]/text()')+
                      etree.HTML(requests.get(i[5],headers=headers).text).xpath('//p[@class="mt-0 mb-xsm v2__EIReviewDetailsV2__bodyColor v2__EIReviewDetailsV2__lineHeightLarge  "]/text()')+
                      etree.HTML(requests.get(i[6],headers=headers).text).xpath('//p[@class="mt-0 mb-xsm v2__EIReviewDetailsV2__bodyColor v2__EIReviewDetailsV2__lineHeightLarge  "]/text()'))

In [153]:
keyword=[]
for comments in all_reviews:
    word_tokens=[word_tokenize(i) for i in comments]
    cleaned=[]
    f_cleaned=[]
    a_cleaned=[]
    vocab=[]
    X=[]
    clean_tokens=[[word for word in i if word.isalnum()]for i in word_tokens]
    for i in clean_tokens:
        for j in i:
            cleaned.append(j)

    for i in cleaned:
        if i in my_stopwords:
            a_cleaned.append(i)
        else:
            f_cleaned.append(i)

    word_count=Counter(f_cleaned)

    for word in word_count.keys():
        if word_count[word]>7:
            vocab.append(word)
    keyword.append(vocab[:5])

In [154]:
keyword

[['pay', 'culture', 'environment', 'balance'],
 ['life', 'balance', 'environment', 'opportunities', 'culture'],
 ['employees', 'environment', 'opportunities', 'pay', 'culture'],
 ['environment', 'pay', 'opportunities'],
 ['balance', 'environment', 'pay', 'growth', 'culture'],
 ['balance', 'environment'],
 ['culture', 'pay', 'growth', 'managers', 'employees'],
 ['pay', 'environment', 'opportunities'],
 ['opportunities', 'growth', 'pay', 'Bell', 'culture'],
 ['pay', 'environment'],
 ['customers', 'food', 'environment', 'busy', 'staff'],
 ['Work', 'home', 'environment', 'salary', 'balance'],
 ['flexible', 'busy', 'food', 'pay', 'experience'],
 ['opportunities', 'experience', 'balance', 'culture', 'projects'],
 ['growth', 'environment', 'culture', 'Management'],
 ['environment', 'customers', 'staff', 'pay', 'employees'],
 ['pay', 'employees', 'environment', 'food'],
 ['environment', 'customers', 'manager', 'flexible', 'really'],
 ['environment', 'experience', 'culture', 'positive', 'home']

In [158]:
selected_re=[]
for i in all_reviews:
    selected_re.append(i[:5])

In [159]:
selected_re

[['yearly bonus and health and dental',
  'it is a larger corporation and little employee excellence recognition.',
  'I am with The Bank for 20 years and TD is a great palce to work',
  "I can't think of any after 20 years",
  'Flexibility, good pay, unlimited earnings'],
 ['Pay, Location, Exit Opportunities and networking',
  'There are None that I can think of',
  'Great exposure to different business units at RBC. Expanded my network a great deal.',
  'Uncompetitive compensation relative to consulting firms or other relevant strategic roles (even for interns).',
  'Many pros such as benefits and good work life balance'],
 ['A super nice office with helpful employees',
  'There were a lot of departments so you had to go through the hoops in order to get access to all thingss',
  'Great people. \nGreat Campus.\nGreat Brand.',
  'Not applicable. I have no cons that come to mind.',
  'Competitive salary for same role'],
 ['Full benefit, respect people and their time, access to saving p

In [161]:
Reviews=pd.DataFrame()
Reviews['Company_name']=companies
Reviews['Recently_Reviews']=selected_re

In [162]:
Reviews

,Company_name,Recently_Reviews
0,TD,"[yearly bonus and health and dental, it is a l..."
1,RBC,"[Pay, Location, Exit Opportunities and network..."
2,Rogers Communications,"[A super nice office with helpful employees, T..."
3,Scotiabank,"[Full benefit, respect people and their time, ..."
4,CIBC,"[Good salary, benefits, flexible work time, wo..."
5,IBM,[Job security for younger generation. Good wo...
6,BMO Financial Group,"[Great job No concerns love it, None to note a..."
7,University of Toronto,"[Great staff, no issues at all, Not applicable..."
8,Bell Canada,"[Great place to work, None that i can think of..."
9,Walmart,"[Best place to work, meet new people, No cons ..."
